# 2.1.2 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
ava = OpenAI(instructions)
parser = Parser()

In [2]:
# Zeichenerklärung – Schema Nutzungsschablone
# SKIP: ze_raw_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-raw.png"
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE.png"
ze_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"

async def run(ze_path):
    base64_ze, img_size = parser.imageToBase64(ze_path)
    print("image_size:", img_size)
    message = ava.request([
        {
            "type": "text",
            "text": 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_ze}",
                "detail": "high"
            }
        },
    ])
    return message

messages = await asyncio.gather(run(ze_raw_path), run(ze_path), run(ze_crop_path))
for message in messages:
    print(message)

image_size: [4625, 8208]
image_size: [768, 1362]
image_size: [768, 1949]
1. **Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung:**

   Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter der Überschrift "Planungsrechtliche Festsetzungen". Er ist durch ein Symbol dargestellt, das aus einem Quadrat mit einem diagonalen Strich besteht.

2. **Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone":**

   Die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone" befindet sich direkt rechts neben dem Symbol und lautet: "Nutzungsschablone (§ 19 (4) BauGB)".

3. **Gebe die Tabellenerklärung in ASCII-Tabellenform aus:**

   Hier ist die Tabellenerklärung in ASCII-Tabellenform:

   ```
   +-------------------+-------------------------------+
   | Symbol            | Erklärung                     |
   +-------------------+-------------------------------+
   | [Quadrat mit      | Nutzungsschablone             |
   | diagonalen 

In [3]:
# Zeichenerklärung – Schema Nutzungsschablone WITH BOUNDING BOX
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg4 = ava.request([
    {
        "type": "text",
        "text": 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg4)

image_size: [763, 2000]
Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+---------------------+------------------+
| Art der Nutzung     | Gebäudehöhe      |
+---------------------+------------------+
| Grundflächenzahl    | Geschossflächen- |
|                     | zahl             |
+---------------------+------------------+
| Bauweise            |                  |
+---------------------+------------------+
```


In [9]:
# Zeichenerklärung – Komplette Definition Nutzungsschablone
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

ava.addContext([
    {
        "type": "text",
        "text": msg4
    }
])

msg5 = ava.request([
    {
        "type": "text",
        "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Beispiel"=""}}'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg5)
ava.clearContext()

image_size: [768, 1949]
Hier sind die relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone im JSON-Format:

```json
{
  "Art der Nutzung": {
    "Definition": "Gewerbegebiete (§ 8 BauNVO) oder Gewerbegebiete eingeschränkt (§ 8 BauNVO)",
    "Beispiel": "GE, GEE"
  },
  "Gebäudehöhe": {
    "Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe",
    "Beispiel": "GH=max.12,00"
  },
  "Grundflächenzahl": {
    "Definition": "Grundflächenzahl (GRZ) gemäß § 16 BauNVO",
    "Beispiel": "0,6"
  },
  "Geschossflächenzahl": {
    "Definition": "Geschossflächenzahl (GFZ) gemäß § 16 BauNVO",
    "Beispiel": "1,2"
  },
  "Bauweise": {
    "Definition": "Abweichende Bauweise gemäß § 22 BauNVO",
    "Beispiel": "a"
  }
}
```
